<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/best_practices/11_yaml_config_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyYaml python-box rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 26.5 MB/s eta 0:00:00


In [2]:
import pooch
import rasterio as rio
import yaml
from box import Box

In [3]:
def generate_difference_image(cfg):

    with rio.open(cfg.input_image) as src:
        src_cfg = src.profile
        b1, b2 = src.read(cfg.band_ixs)
        result = (b2 - b1) / (b2 + b1)

    src_cfg.update(count=1)
    with rio.open(cfg.output_image, 'w', **src_cfg) as out:
        out.write(result, 1)

In [4]:
sentinel2_img = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/main/raster/oromocto_jun2021_composite.tif',
    known_hash='e08df41e0f301d684f9ed1a487b9abccaf431fd3df25fa5f10da82d44e6b733d'
)

In [5]:
cfg = yaml.safe_load(f'''

input_image: {sentinel2_img}
output_image: ndvi.tif
band_ixs:
  - 3
  - 7

''')

print(cfg)

{'input_image': '/root/.cache/pooch/1b7989bb99434afca6e9eaa37ae6579a-oromocto_jun2021_composite.tif', 'output_image': 'ndvi.tif', 'band_ixs': [3, 7]}


In [6]:
generate_difference_image(Box(cfg))

# Full Code

In [ ]:
"""
Vaasudevan Srinivasan
StratagemGIS Solutions 🧑🏻‍💻
Script to read yaml as config file and generate ndvi
"""


# !pip install PyYaml python-box rasterio

# config.yaml
'''
input_image: oromocto_jun2021_composite.tif
output_image: ndvi.tif
band_ixs:
  - 3
  - 7
'''


import rasterio as rio
import yaml
from box import Box


def generate_difference_image(cfg):

    with rio.open(cfg.input_image) as src:
        src_cfg = src.profile
        b1, b2 = src.read(cfg.band_ixs)
        result = (b2 - b1) / (b2 + b1)

    src_cfg.update(count=1)
    with rio.open(cfg.output_image, 'w', **src_cfg) as out:
        out.write(result, 1)


with open('config.yaml') as f:

    cfg_content = yaml.safe_load(f)

    # print(cfg_content)
    # {'input_image': 'oromocto_jun2021_composite.tif',
    #  'output_image': 'ndvi.tif',
    #  'band_ixs': [3, 7]}

    # Box allows to access dict values using dot notation
    generate_difference_image(Box(cfg_content))
